In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import plot_model

In [134]:
temp = pd.read_csv("obesity.csv")
columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
for c in columns:
  dummies = pd.get_dummies(temp[c], prefix=(str(c)+"_"))
  temp = pd.concat([temp, dummies], axis=1)
temp = temp.drop(columns=columns)


d = ['Insufficient_Weight',
       'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III']
ordinal_encoder = OrdinalEncoder(categories=[d])
temp['NObeyesdad'] = ordinal_encoder.fit_transform(temp[['NObeyesdad']])
y_xgb = temp['NObeyesdad']

temp = temp.drop('NObeyesdad', axis=1)
X_train, X_test, y_train, y_test = train_test_split(temp, y_xgb, test_size=0.2)

In [135]:
print(y_xgb)

0       1.0
1       1.0
2       1.0
3       2.0
4       3.0
       ... 
2106    6.0
2107    6.0
2108    6.0
2109    6.0
2110    6.0
Name: NObeyesdad, Length: 2111, dtype: float64


In [136]:
print(temp['NCP'].value_counts())

3.000000    1203
1.000000     199
4.000000      69
2.776840       2
3.985442       2
            ... 
3.054899       1
3.118013       1
3.335876       1
3.205009       1
1.089048       1
Name: NCP, Length: 635, dtype: int64


In [137]:
model = XGBClassifier(objective='multi:softprob')

In [138]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [139]:
y_pred = model.predict(X_test)
y_pred

array([2, 1, 3, 2, 4, 0, 6, 6, 5, 6, 6, 2, 1, 5, 4, 2, 5, 3, 0, 1, 4, 0,
       6, 4, 4, 2, 3, 6, 6, 1, 2, 4, 3, 5, 3, 2, 3, 6, 1, 0, 5, 1, 3, 0,
       0, 2, 5, 1, 3, 6, 0, 0, 6, 1, 6, 5, 5, 3, 4, 4, 2, 0, 1, 6, 6, 3,
       5, 2, 1, 2, 6, 4, 5, 3, 5, 0, 6, 4, 4, 2, 0, 2, 6, 0, 1, 4, 1, 5,
       6, 1, 0, 4, 4, 1, 1, 3, 0, 1, 5, 5, 5, 4, 3, 2, 6, 5, 6, 5, 3, 1,
       6, 5, 1, 4, 0, 2, 2, 3, 6, 4, 3, 4, 2, 6, 5, 0, 2, 6, 4, 0, 1, 2,
       0, 4, 0, 1, 4, 1, 4, 3, 6, 3, 4, 5, 2, 2, 3, 6, 0, 2, 4, 0, 5, 1,
       4, 2, 6, 0, 5, 0, 2, 4, 6, 3, 5, 5, 0, 2, 3, 2, 5, 4, 3, 5, 3, 6,
       0, 3, 5, 4, 3, 6, 5, 3, 6, 0, 3, 6, 4, 4, 5, 0, 3, 4, 1, 0, 1, 2,
       5, 5, 4, 4, 4, 3, 5, 2, 3, 2, 1, 3, 1, 0, 1, 6, 2, 0, 0, 3, 3, 3,
       6, 6, 5, 0, 0, 4, 6, 3, 3, 1, 6, 1, 6, 4, 5, 0, 3, 1, 0, 4, 3, 3,
       1, 4, 1, 2, 0, 6, 2, 4, 2, 4, 5, 2, 5, 1, 2, 3, 0, 4, 4, 2, 3, 2,
       5, 1, 6, 0, 6, 2, 4, 5, 6, 3, 0, 6, 0, 3, 5, 4, 0, 4, 6, 5, 6, 6,
       2, 1, 4, 6, 3, 6, 1, 0, 2, 1, 2, 5, 0, 6, 1,

In [140]:
y_pred_proba = model.predict_proba(X_test)
print(y_pred_proba)

[[1.3789101e-04 7.9292618e-04 9.9676239e-01 ... 6.4204854e-04
  2.4325530e-04 3.3575695e-04]
 [7.5607188e-04 9.8020387e-01 1.3911975e-02 ... 1.3817415e-03
  1.1231700e-03 7.5292320e-04]
 [5.7010657e-05 6.7876803e-04 1.6057072e-03 ... 1.1859569e-04
  8.2413673e-05 1.2825563e-04]
 ...
 [2.0752937e-05 9.4296047e-06 2.6897873e-05 ... 2.7310941e-04
  9.9954504e-01 9.4778348e-05]
 [9.9493235e-01 3.1585505e-03 7.6802494e-04 ... 3.2985656e-04
  1.5524690e-04 1.9694869e-04]
 [1.9768088e-03 8.5062164e-01 1.4563359e-01 ... 6.5872952e-04
  1.9024413e-04 4.7114174e-04]]


In [141]:
accuracy = model.score(X_test, y_test)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

Accuracy: 97.8723%


In [142]:
#XGB Cross Validation - https://machinelearningmastery.com/evaluate-gradient-boosting-models-xgboost-python/
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
def createXGB():
  model = XGBClassifier(objective='multi:softprob', enable_categorical=True)
  return model
def train_evaluateXGB(model, xtrain, ytrain, xtest, ytest):
  model.fit(xtrain, ytrain)
  accuracy = model.score(xtest, ytest)
  return accuracy
kFold = StratifiedKFold(n_splits = 10)
i=0
sum=0
for train, test in kFold.split(temp,y_xgb):
  print("Current Iteration:",i)
  modelxgb = None
  modelxgb = createXGB()

  acc = train_evaluateXGB(modelxgb,temp.loc[[x for x in train]],y_xgb.loc[[x for x in train]],temp.loc[[x for x in test]],y_xgb.loc[[x for x in test]])
  i+=1
  sum += acc

print("Mean accuracy:", (sum/i))
#Mean accuracy: 0.96499150496289

Current Iteration: 0


Current Iteration: 1
Current Iteration: 2
Current Iteration: 3
Current Iteration: 4
Current Iteration: 5
Current Iteration: 6
Current Iteration: 7
Current Iteration: 8
Current Iteration: 9
Mean accuracy: 0.96499150496289


In [143]:
import pickle

In [149]:
filename = 'xgb_model.sav'
pickle.dump(model, open(filename, 'wb'))


In [151]:
xgb_model = pickle.load(open('xgb_model.sav', 'rb'))